### Imports

In [1]:
import os
import os.path
import json

from scipy.optimize import linear_sum_assignment

import numpy as np

import glob

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib

import math

import collections

Function to convert bbox to range [0, 1]

In [2]:
def convert_bbox_to_coordinates(bbox, HH, WW):
    """
    input: bbox of type (x, y, w,h)
    
    output: bbox of type (x, y, x1, x2)
    """
    # Convert to range [0, 1] 
    x, y, w, h = bbox
    x0 = x / WW
    y0 = y / HH
    x1 = (x + w) / WW
    y1 = (y + h) / HH
    return [x0, y0, x1, y1]

In [3]:
def clamp(x, min_val, max_val):
    """
    Function to clamp a value
    """
    if x < min_val:
        return 0
    elif x > max_val:
        return 1
    else:
        return x

In [4]:
def convert_bbox_mean_to_coordinates(bbox, HH, WW):
    # Convert to range [0, 1]
    """
    input: bbox of type (x_center, y_center, w,h)
    
    output: bbox of type (x, y, x1, x2)
    """
    x, y, w, h = bbox
    x0 = (x - w/2) / WW
    y0 = (y - h/2) / HH
    x1 = (x0 + w) / WW
    y1 = (y0 + h) / HH
    
    x0 = clamp(x0, 0, 1)
    x1 = clamp(x1, 0, 1)
    y0 = clamp(y0, 0, 1)
    y1 = clamp(y1, 0, 1)
    
    return [x0, y0, x1, y1]

### Metrics

IoU@P and IoU@R

In [5]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    if xA == xB or yA == yB:
        return 1
    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

# Precision helper function
def precision_at(threshold, iou):
    # TP: A correct detection. Detection with IOU ≥ threshold
    # FP: A wrong detection. Detection with IOU < threshold
    # FN: A ground truth not detected
    true_positives, false_positives, false_negatives = 0, 0, 0
    for i in iou:
        if i >= threshold:
            true_positives += 1
        else:
            false_positives += 1
    return true_positives, false_positives, false_negatives

def precision(iou, name, version, epoch, fn, print_table=True):
    # Loop over IoU thresholds
    if not os.path.exists("./metrics/IoUAndRecall/"):
        os.mkdir("./metrics/IoUAndRecall/")
    
    if not os.path.exists("./metrics/IoUAndRecall/"+str(version)+"/"):
        os.mkdir("./metrics/IoUAndRecall/"+str(version)+"/")
    
        
    with open("./metrics/IoUAndRecall/"+str(version)+"/" + str(epoch) + "precision.txt", "w") as f:
        prec = []
        if print_table:
            print("Thresh\tTP\tFP\tFN\tPrec.")
        f.write("Thresh\tTP\tFP\tFN\tPrec.\n")
        for t in np.arange(0.3, 1.0, 0.05):
            tp, fp, _ = precision_at(t, iou)
            if (tp + fp) > 0:
                p = tp / (tp + fp)
            else:
                p = 0
            if print_table:
                print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
            f.write("{:1.3f}\t{}\t{}\t{}\t{:1.3f}\n".format(t, tp, fp, fn, p))
            prec.append(p)

        if print_table:
            print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
        f.write("AP\t-\t-\t-\t{:1.3f}\n".format(np.mean(prec)))
    return np.mean(prec)

def recall(iou, name, version, epoch, fn, print_table=True):
    # Loop over IoU thresholds
    if not os.path.exists("./metrics/IoUAndRecall/"):
        os.mkdir("./metrics/IoUAndRecall/")
    
    if not os.path.exists("./metrics/IoUAndRecall/"+str(version)+"/"):
        os.mkdir("./metrics/IoUAndRecall/"+str(version)+"/")
    

    with open("./metrics/IoUAndRecall/"+str(version)+"/" + str(epoch) + "recall.txt", "w") as f:
        prec = []
        if print_table:
            print("Thresh\tTP\tFP\tFN\tPrec.")
        f.write("Thresh\tTP\tFP\tFN\tPrec.\n")
        for t in np.arange(0.3, 1.0, 0.05):
            tp, fp, _ = precision_at(t, iou)
            if (tp + fn) > 0:
                p = tp / (tp + fn)
            else:
                p = 0
            if print_table:
                print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
            f.write("{:1.3f}\t{}\t{}\t{}\t{:1.3f}\n".format(t, tp, fp, fn, p))
            prec.append(p)

        if print_table:
            print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
        f.write("AP\t-\t-\t-\t{:1.3f}\n".format(np.mean(prec)))
    return np.mean(prec)

Relative spatial position categorical

In [6]:
def compute_relative_spatial_position_categorical(bbox_gt, bbox_pred, verbose=False):
    def compute_obj_centers(bbox1):
        x0, y0, x1, y1 = bbox1
        return (0.5 * (x0 + x1), 0.5 * (y0 + y1))
    
    def relative_spatial_position_categorical(bbox1, bbox2):
        # Check for inside / surrounding
        sx0, sy0, sx1, sy1 = bbox1
        ox0, oy0, ox1, oy1 = bbox2
        d0, d1 = compute_obj_centers(bbox1), compute_obj_centers(bbox2)
        d3 = (d0[0] - d1[0], d0[1]-d1[1])
        theta = math.atan2(d3[1], d3[0])

        if theta >= 3 * math.pi / 4 or theta <= -3 * math.pi / 4:
            p = 'left of'
        elif -3 * math.pi / 4 <= theta < -math.pi / 4:
            p = 'above'
        elif -math.pi / 4 <= theta < math.pi / 4:
            p = 'right of'
        elif math.pi / 4 <= theta < 3 * math.pi / 4:
            p = 'below'
        return p
    
    def check_relative_spatial_positions_categorical(bbox):
        rel_spa_pos = []
        for i in range(len(bbox)):
            for j in range(len(bbox)):
                if i != j:
                    out = relative_spatial_position_categorical(bbox[i], bbox[j])
                    rel_spa_pos.append(out)
        return rel_spa_pos
    
    def compare_relative_spatial_positions_categorical(gt, pred):
        acc = 0
        for i in range(len(gt)):
            if gt[i] == pred[i]:
                acc += 1
            elif (gt[i] == "left of" and pred[i] == "right of") or (gt[i] == "right of" and pred[i] == "left of"):
                acc += 1
        
        return acc/len(gt) if len(gt) > 0 else 0
    
    rsp_cat = check_relative_spatial_positions_categorical(bbox_gt)
    rsp2_cat = check_relative_spatial_positions_categorical(bbox_pred)
    if verbose:
        print(rsp_cat, rsp2_cat)
    return compare_relative_spatial_positions_categorical(rsp_cat, rsp2_cat)

Relative spatial position numerical

In [7]:
def compute_relative_spatial_position_numerical(bbox_gt, bbox_pred, verbose=False):
    def compute_obj_centers(bbox1):
        x0, y0, x1, y1 = bbox1
        return (0.5 * (x0 + x1), 0.5 * (y0 + y1))
    
    def relative_spatial_position_numerical(bbox1, bbox2):
        # Check for inside / surrounding
        sx0, sy0, sx1, sy1 = bbox1
        ox0, oy0, ox1, oy1 = bbox2
        d0, d1 = compute_obj_centers(bbox1), compute_obj_centers(bbox2)
        d3 = (d0[0] - d1[0], d0[1]-d1[1])
        return math.atan2(d3[1], d3[0])
    
    def check_relative_spatial_positions_numerical(bbox):
        rel_spa_pos = []
        for i in range(len(bbox)):
            for j in range(len(bbox)):
                if i != j:
                    out = relative_spatial_position_numerical(bbox[i], bbox[j])
                    rel_spa_pos.append(out)
        return rel_spa_pos
    
    def compare_relative_spatial_positions_numerical(gt, pred):
        acc = 0
        for i in range(len(gt)):
            acc += abs(gt[i] - pred[i])

        # print(acc_gt, acc_pred)
        return acc/len(gt) if len(gt) > 0 else 0
    
    rsp_cat = check_relative_spatial_positions_numerical(bbox_gt)
    rsp2_cat = check_relative_spatial_positions_numerical(bbox_pred)
    if verbose:
        print(rsp_cat, rsp2_cat)
    return compare_relative_spatial_positions_numerical(rsp_cat, rsp2_cat)

Aspect ratio

In [8]:
def compute_aspect_ratio(bbox_gt, bbox_pred, verbose=False):
    def check_aspect_ratio(gt, pred, verbose=False):
        acc = 0
        for i in range(len(gt)):
            gt_x1, gt_y1, gt_x2, gt_y2 = gt[i]
            pr_x1, pr_y1, pr_x2, pr_y2 = pred[i]
            
            theta_gt = np.arctan2(gt_y2 - gt_y1, gt_x2 - gt_x1)
            theta_pr = np.arctan2(pr_y2 - pr_y1, pr_x2 - pr_x1)
            
            acc += abs(theta_gt - theta_pr)
    
        return acc/len(gt) if len(gt) > 0 else 0
    
    return check_aspect_ratio(bbox_gt, bbox_pred, False)

Class matching

In [9]:
def compute_class_matching(obj_gt, obj_pred):
    def class_matching(gt, pred):
        gt_counter = {}
        pred_counter = {}
        for i in gt:
            if i in gt_counter:
                gt_counter[i] += 1
            else:
                gt_counter[i] = 1
        for i in pred:
            if i in pred_counter:
                pred_counter[i] += 1
            else:
                pred_counter[i] = 1

        TP, FP, FN = 0, 0, 0

        for key in set(list(gt_counter.keys()) + list(pred_counter.keys())):
            if key in pred_counter and key in gt_counter:
                diff = gt_counter[key] - pred_counter[key]
                if diff == 0:
                    TP += gt_counter[key]
                else:
                    TP += min(gt_counter[key], pred_counter[key])
                    if diff < 0:
                        FP += abs(diff)
                    else:
                        FN += abs(diff)
            elif key in gt_counter:
                FN += gt_counter[key]
            else:
                FP += pred_counter[key]
        recall = TP/(TP+FN) if TP+FN > 0 else 0
        precision = TP/(TP+FP) if TP+FP > 0 else 0
        f1_score = (2*recall*precision)/(recall+precision) if recall + precision != 0 else 0
        return recall, precision, f1_score
    return class_matching(obj_gt, obj_pred)
        

Relative scale

In [10]:
def compute_relative_scale(bbox_gt, bbox_pred):
    def relative_scale(gt, pred, verbose=False, sol1=False):
        acc, total = 0, 0
        for i in range(len(gt)):
            for j in range(len(gt)):
                if i != j:
                    gt1_x1, gt1_y1, gt1_x2, gt1_y2 = gt[i]
                    gt2_x1, gt2_y1, gt2_x2, gt2_y2 = gt[j]

                    pr1_x1, pr1_y1, pr1_x2, pr1_y2 = pred[i]
                    pr2_x1, pr2_y1, pr2_x2, pr2_y2 = pred[j]

                    gt1_height, gt1_width = gt1_y2 - gt1_y1, gt1_x2 - gt1_x1
                    gt2_height, gt2_width = gt2_y2 - gt2_y1, gt2_x2 - gt2_x1

                    pred1_height, pred1_width = pr1_y2 - pr1_y1, pr1_x2 - pr1_x1
                    pred2_height, pred2_width = pr2_y2 - pr2_y1, pr2_x2 - pr2_x1

                    gt1_area = gt1_height * gt1_width
                    gt2_area = gt2_height * gt2_width

                    pred1_area = pred1_height * pred1_width
                    pred2_area = pred2_height * pred2_width
                    
                    if gt1_area == 0 or gt2_area == 0 or pred1_area == 0 or pred2_area == 0:
                        continue
                    R1, R2 = pred1_area/pred2_area, gt1_area/gt2_area, 

                    acc += abs(R1-R2)

                    total += 1
        return acc/total if total > 0 else 0

    return relative_scale(bbox_gt, bbox_pred)

In [11]:
def generate_metrics(gt, pred, coco_objects_gt, coco_objects_pred, verbose=False):
    # print(gt, pred, coco_objects_gt, coco_objects_pred)
    rspc, rspn, ar, rs, r, p, f = 0, 0, 0, 0, 0, 0, 0
    # RSPC
    rspc = compute_relative_spatial_position_categorical(gt, pred, verbose)
        
    # RSPN
    rspn = compute_relative_spatial_position_numerical(gt, pred, verbose)
        
    # AR
    ar = compute_aspect_ratio(gt, pred, verbose)
        
    # CM
    r, p, f = compute_class_matching(coco_objects_gt, coco_objects_pred)
                   
    # RS
    rs = compute_relative_scale(gt, pred)
    
    return rspc, rspn, ar, rs, r, p, f

### Coco validation

In [12]:
COCO_VAL_ANNOT = "../TRAN2LY/data/datasets/COCO/annotations/instances_val2014.json"

In [13]:
with open(COCO_VAL_ANNOT, "r") as json_file:
    coco = json.load(json_file)

In [14]:
objects = {}

In [15]:
# Obtain objects
for ann in coco['annotations']:
    image_id = str(ann['image_id'])
    if image_id in objects:
        objects[image_id]['ann'].append(ann)
    else:
        objects[image_id] = {'ann': [ann]}

# Obtain image width and height
for images in coco['images']:
    image_id = str(images['id'])
    if image_id in objects:
        objects[image_id]['size'] = (images["height"], images['width'])

In [16]:
#objects.keys()

### Check the metrics

In [17]:
VERSION = "carlos_pt"
MODEL = "RNN2LY"
files = list(glob.glob("../"+MODEL+"/evaluator_output/"+VERSION+"/*.json"))
# files = list(glob.glob("../TRAN2LY/evaluator_output/"+str(VERSION)+"/*.json"))
files.sort(key=lambda x: (len(x), x))

In [18]:
files = [f for f in files if "TESTING" in f]
files

['../RNN2LY/evaluator_output/carlos_pt\\TESTINGepoch27.json']

In [19]:
len(files)

1

In [20]:
all_metrics = {}

In [21]:
def generate_metrics_epoch(file_name, epoch):
    not_found = 0
    with open(file_name, "r") as json_file:
        prediction = json.load(json_file)
        
        false_negatives, iou, extra_objects = 0, [], 0
        output = {
            "rspc": [],
            "rspn": [],
            "ar": [],
            "rs": [],
            "r": [],
            "f": [],
            "p": [],
            "extra_objects": []
        }
        for key, value in prediction.items():
            if key.split("-")[0] not in objects:
                not_found += 1
                continue
            orig_objs = objects[key.split("-")[0]]['ann']
            orig_size = objects[key.split("-")[0]]['size']
            pred_objs = value
       
            orig_pred = {i:{'pred':[], 'orig': []} for i in range(1, 92)}
            for obj in orig_objs:
                out = convert_bbox_to_coordinates(obj['bbox'], orig_size[0], orig_size[1])
                orig_pred[obj['category_id']]['orig'].append(out)

            for obj in pred_objs:
                bbox, ls = obj[:4], obj[-1]
                out = convert_bbox_mean_to_coordinates(bbox, 1, 1)
                orig_pred[int(ls)]['pred'].append(out)
                
            bbox_gt, bbox_pred, coco_objects_gt, coco_objects_pred = [], [], [], []

            for category_id in orig_pred.keys():
                s = 0
                orig_objs_match = orig_pred[category_id]['orig']
                pred_objs_match = orig_pred[category_id]['pred']
                
                if len(orig_objs_match) == 0:
                    output['extra_objects'].append(len(pred_objs_match))
                    continue
                elif len(pred_objs_match) == 0:
                    coco_objects_gt += (len(orig_objs_match)*[category_id])
                    continue

                coco_objects_gt += (len(orig_objs_match)*[category_id])
                coco_objects_pred += (len(pred_objs_match)*[category_id])

                dp = [[1]*len(pred_objs_match) for i in range(len(orig_objs_match))]
                # calculate the matrix of ious
                for i in range(len(orig_objs_match)):
                    for j in range(len(pred_objs_match)):
                        # print(orig_objs_match[i], pred_objs_match[j])
                        dp[i][j] = bb_intersection_over_union(orig_objs_match[i], pred_objs_match[j])

                cost = np.array(dp, dtype=np.float64)
                row_ind, col_ind = linear_sum_assignment(cost, maximize=True)
                for col_ind, row_ind in zip(list(col_ind), list(row_ind)):
                    iou.append(dp[row_ind][col_ind])
                    bbox_gt.append(orig_objs_match[row_ind])
                    bbox_pred.append(pred_objs_match[col_ind])
                    s += 1
                false_negatives += abs(len(orig_objs_match) - s)
            
            only_rpf = False
            if len(bbox_gt) == 0 and len(bbox_pred) == 0:
                only_rpf = True
            rspc, rspn, ar, rs, r, p, f = generate_metrics(bbox_gt, bbox_pred, coco_objects_gt, coco_objects_pred, False)
            if not only_rpf:
                output['rspc'].append(rspc)
                output['rspn'].append(rspn)
                output['ar'].append(ar)
                output['rs'].append(rs)
            output['r'].append(r)
            output['p'].append(p)
            output['f'].append(f)
    print("NOT FOUND", not_found)       
    precision(iou, "test", str(VERSION), str(epoch), false_negatives)
    recall(iou, "test", str(VERSION), str(epoch), false_negatives)
    real_output = {
        "rspc": sum(output['rspc'])/len(output['rspc']) if len(output['rspc']) > 0 else 0,
        "rspn": sum(output['rspn'])/len(output['rspn']) if len(output['rspn']) > 0 else 0,
        "ar": sum(output['ar'])/len(output['ar']) if len(output['ar']) > 0 else 0,
        "rs": sum(output['rs'])/len(output['rs']) if len(output['rs']) > 0 else 0,
        "r": sum(output['r'])/len(output['r']) if len(output['r']) > 0 else 0,
        "f": sum(output['f'])/len(output['f']) if len(output['f']) > 0 else 0,
        "p": sum(output['p'])/len(output['p']) if len(output['p']) > 0 else 0,
        "extra_objects": sum(output['extra_objects'])
    } 
    all_metrics[epoch] = real_output
    print(all_metrics[epoch])
    print(not_found)

In [22]:
if not os.path.exists("./metrics/"):
    os.mkdir("./metrics/")
        
epoch = 0
for file in files:
    print("Calculating metrics for file:", file)
    generate_metrics_epoch(file, epoch)
    epoch += 1


Calculating metrics for file: ../RNN2LY/evaluator_output/carlos_pt\TESTINGepoch27.json
NOT FOUND 0
Thresh	TP	FP	FN	Prec.
0.300	8773	32647	39523	0.212
0.350	6481	34939	39523	0.156
0.400	4568	36852	39523	0.110
0.450	3047	38373	39523	0.074
0.500	1883	39537	39523	0.045
0.550	1114	40306	39523	0.027
0.600	627	40793	39523	0.015
0.650	301	41119	39523	0.007
0.700	143	41277	39523	0.003
0.750	50	41370	39523	0.001
0.800	20	41400	39523	0.000
0.850	4	41416	39523	0.000
0.900	0	41420	39523	0.000
0.950	0	41420	39523	0.000
AP	-	-	-	0.047
Thresh	TP	FP	FN	Prec.
0.300	8773	32647	39523	0.182
0.350	6481	34939	39523	0.141
0.400	4568	36852	39523	0.104
0.450	3047	38373	39523	0.072
0.500	1883	39537	39523	0.045
0.550	1114	40306	39523	0.027
0.600	627	40793	39523	0.016
0.650	301	41119	39523	0.008
0.700	143	41277	39523	0.004
0.750	50	41370	39523	0.001
0.800	20	41400	39523	0.001
0.850	4	41416	39523	0.000
0.900	0	41420	39523	0.000
0.950	0	41420	39523	0.000
AP	-	-	-	0.043
{'rspc': 0.4747210429432272, 'rspn': 1.16215480

In [23]:
#generate RS softmax
rs_values = []
for key, value in all_metrics.items():
    rs_values.append(value['rs'])
rs_softmaxed = np.exp(rs_values)/np.sum(np.exp(rs_values))
for i,key in enumerate(all_metrics.keys()):
    all_metrics[key]['rs_softmax'] = rs_softmaxed[i]

In [24]:
import glob
from openpyxl import Workbook

In [25]:
workbook = Workbook()
sheet = workbook.active

In [26]:
index_to_char = {i+1:j for i, j in enumerate(list("ABCDEFGHIJKLMNOPQRSTVWXYZ"))}

precision and recall

In [27]:
files = glob.glob("./metrics/IoUAndRecall/" + str(VERSION) + "/*.txt")
files.sort(key=lambda x: ("precision" in x, len(x), x))

# The first 10 precision the next 10 recall

In [28]:
files

['./metrics/IoUAndRecall/carlos_pt\\0recall.txt',
 './metrics/IoUAndRecall/carlos_pt\\0precision.txt']

In [29]:
k = 1
for file in files:
    print(file)
    j = 1
    with open(file, "r") as data:
        data.readline()
        for line in data:
            identificator = index_to_char[j] + str(k)
            print(identificator)
            sheet[identificator] = str(float(line.split("\t")[-1][:-1])).replace(".", ",")
            # Change character (column)
            j += 1
    # Change number (row)
    k += 1

./metrics/IoUAndRecall/carlos_pt\0recall.txt
A1
B1
C1
D1
E1
F1
G1
H1
I1
J1
K1
L1
M1
N1
O1
./metrics/IoUAndRecall/carlos_pt\0precision.txt
A2
B2
C2
D2
E2
F2
G2
H2
I2
J2
K2
L2
M2
N2
O2


In [30]:
workbook.save(filename="./metrics/IoUAndRecall/" + str(VERSION) + "/precision_and_recall.xlsx")

Remaining metrics

In [31]:
workbook = Workbook()
sheet = workbook.active

In [32]:
for key, value in all_metrics.items():
    sheet["A"+str(key+2)] = str(value["rspc"]).replace(".", ",")
    sheet["B"+str(key+2)] = str(value["rspn"]).replace(".", ",")
    sheet["C"+str(key+2)] = str(value["ar"]).replace(".", ",")
    sheet["D"+str(key+2)] = str(value["rs"]).replace(".", ",")
    sheet["E"+str(key+2)] = str(value["rs_softmax"]).replace(".", ",")
    sheet["F"+str(key+2)] = str(value["p"]).replace(".", ",")
    sheet["G"+str(key+2)] = str(value["f"]).replace(".", ",")
    sheet["H"+str(key+2)] = str(value["r"]).replace(".", ",")
    sheet["I"+str(key+2)] = str(value["extra_objects"]).replace(".", ",")
    sheet["J"+str(key+2)] = "=A"+str(key+2)+"+1-C"+str(key+2)+"+1-E"+str(key+2)+"+F"+str(key+2)+"+G"+str(key+2)+"+H"+str(key+2)
sheet["A1"] = "rspc"
sheet["B1"] = "rspn"
sheet["C1"] = "ar"
sheet["D1"] = "rs"
sheet["E1"] = "rs_softmax"
sheet["F1"] = "p"
sheet["G1"] = "f"
sheet["H1"] = "r"
sheet["I1"] = "extra_objects"
sheet["J1"] = "score"

In [33]:
if not os.path.exists("./metrics/general/"):
    os.mkdir("./metrics/general/")

if not os.path.exists("./metrics/general/" + MODEL+str(VERSION) +"/"):
    os.mkdir("./metrics/general/" + MODEL+str(VERSION) +"/")

In [34]:
workbook.save(filename="./metrics/general/" + MODEL+str(VERSION) + "/"+MODEL+VERSION+"_test.xlsx")